In [15]:
# Importing libraries
import argparse
import re
import os
import random
from glob import glob

from datasets import load_metric
from datasets import Dataset, DatasetDict
import numpy as np
import pandas as pd
from setproctitle import setproctitle
from tqdm import tqdm, trange

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer,T5TokenizerFast, T5ForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# for kobert
# from kobert_tokenizer import KoBERTTokenizer
# from transformers import BertModel

import torch
from kss import split_sentences
import Levenshtein as Lev

import nltk
# nltk.download('punkt')
import string
from transformers import AutoTokenizer

In [28]:
parser = argparse.ArgumentParser()
# parser.add_argument('--dataset_path', default='/gec/data/v1.0/clean_data_v2_utf8.tsv')
# parser.add_argument('--dataset_path', default='/home/gec_sj/gec/data/v1.0/Korean_corpus_2021.csv')
# parser.add_argument('--north_phonetic_data_path', default='/home/gec_sj/gec/data/v1.0/north_corpus.tsv')
# parser.add_argument('--modu_spelling_data_path', default='/home/gec_sj/gec/data/v1.0/Korean_corpus_2021.tsv')
# parser.add_argument('--kspon_spelling_data_path', default='/home/gec_sj/gec/data/v1.0/kspon_corpus.tsv')

# parser.add_argument('--dataset_format', default='tsv')
# parser.add_argument('--pretrained_model', default='results/ke-t5-large-ko_n1_24410개_len512/checkpoint-7000_0.8649_0.970963_0.479547')
# parser.add_argument('--pretrained_model', default='results/ke-t5-large-ko_n_modu_s_670000개/checkpoint-35000_0.5552_0.601422_0.077056')
# parser.add_argument('--pretrained_model', default='results/ke-t5-large-ko_n1_n2_31000개_len64/checkpoint-14000_0.7776_0.704871_0.160412')
# parser.add_argument('--pretrained_model', default='results/ke-t5-large-ko_n1_n2_60000개_len128/checkpoint-8000_0.4808_0.453248_0.420892')
parser.add_argument('--pretrained_model', default='pko_save/len96-checkpoint-110000')
# parser.add_argument('--pretrained_model', default='pko_save/pko-5gram/checkpoint-12000')

parser.add_argument('--max_input_length', default=96)

# parser.add_argument('--pretrained_model', default='KETI-AIR/ke-t5-large-ko')
# parser.add_argument('--pretrained_model', default='skt/kobert-base-v1')
# parser.add_argument('--pretrained_model', default='google/mt5-large')
parser.add_argument('--seed', default=68)
parser.add_argument('--process_name', default='t5_infer_mario')
parser.add_argument('--batch_size', default=32)
parser.add_argument('--devices', default='2,3')
parser.add_argument('--data_limit', default=False)
parser.add_argument('--test_size_ratio', default=0.01)
parser.add_argument('--output_dir', default= "./pko_save")

args = parser.parse_args(args=[])

random.seed(args.seed)

In [29]:
setproctitle(args.process_name)
## cuda device 설정
os.environ["CUDA_VISIBLE_DEVICES"]= args.devices
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)  # 출력결과: cuda 
print('Count of using GPUs:', torch.cuda.device_count())   #출력결과: 2 (2, 3 두개 사용하므로)
print('Current cuda device:', torch.cuda.current_device())  # 출력결과: 2 (2, 3 중 앞의 GPU #2 의미)

Device: cuda
Count of using GPUs: 2
Current cuda device: 0


In [30]:
model_checkpoint = args.pretrained_model
# tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
tokenizer = T5TokenizerFast.from_pretrained(model_checkpoint) # pko 전용

model = T5ForConditionalGeneration.from_pretrained(model_checkpoint).to(device)
# model = model.to(device)

In [ ]:
# """추론 테스트 풀코드"""

# gram_num = 5


# def no_overlap(text_arr,gram_num): # 숫자는 gram 수 대로
#     overlap_text_arr = []
    
#     if len(text_arr)> 0:
#         n = len(text_arr)+1
#     else:
#         n = 1
        
        
#     for j in range(n):
#         if j == 0:
#             tmp_res = text_arr[:gram_num]
#             overlap_text_arr.append(" ".join(tmp_res))
#         else:
#             if j != n - 1:
#                 tmp_res = text_arr[j * gram_num: j * gram_num + gram_num]
#                 overlap_text_arr.append(" ".join(tmp_res))
#             else:
#                 tmp_res = text_arr[j * gram_num:]
#                 overlap_text_arr.append(" ".join(tmp_res))  
#         # print(tmp_res)

#     return overlap_text_arr

# # ---------------------여러 문장일경우-----------
# from kss import split_sentences
# # file_name='kor_data6.txt'
# # text=pd.read_table(f's-kr/fine-tune/dataset/sbs/{file_name}')

# # text=text.columns[0]
# # split_sentences(text)
# # text_arr=text.split(' ')

# # -----문장이 한 문장일경우-----

# # text = '교과서에 나와 있는 지문이 이것보다 훨씬 길고 이 앞에 작품이 하나 더 있어요 그죠'
# # split_sentences(text)
# # text_arr=text.split(' ')

# # -----------------걍 한 컬럼별로 문장이 존재할경우 ------------

# # file_name='kor_data6.txt'
# # text=pd.read_table(f's-kr/fine-tune/dataset/sbs/{file_name}')


# # -----------추론 시작 -------------
# overlap_text_arr = no_overlap(text_arr,gram_num)
# overlap_text_arr = list(dict.fromkeys(overlap_text_arr)) # 중복제거
# overlap_text_arr.remove('')
# # print(overlap_text_arr)


# inf_overlap_text_arr = []
# for text in tqdm(overlap_text_arr):
#     inputs = text
#     inputs = tokenizer([inputs], max_length=args.max_input_length, truncation=True, return_tensors="pt").to(device)
#     output = model.generate(**inputs, num_beams=4, do_sample=False, max_length=args.max_input_length)
#     # inputs = tokenizer([inputs], truncation=True, padding='max_length', max_length=args.max_input_length, return_tensors="pt").to(device)
#     # output = model.generate(**inputs, max_length=args.max_input_length, num_beams=4, num_return_sequences=2, temperature=1.5, do_sample=True)

#     decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
#     inf_overlap_text_arr.append(decoded_output)
#     # print(decoded_output)

# result=' '.join(inf_overlap_text_arr)
# result

In [12]:
"""Ngram-추론 테스트 풀코드"""

gram_num = 5


def no_overlap(text_arr,gram_num): # 숫자는 gram 수 대로
    overlap_text_arr = []
    
    if len(text_arr)> 0:
        n = len(text_arr)+1
    else:
        n = 1
        
        
    for j in range(n):
        if j == 0:
            tmp_res = text_arr[:gram_num]
            overlap_text_arr.append(" ".join(tmp_res))
        else:
            if j != n - 1:
                tmp_res = text_arr[j * gram_num: j * gram_num + gram_num]
                overlap_text_arr.append(" ".join(tmp_res))
            else:
                tmp_res = text_arr[j * gram_num:]
                overlap_text_arr.append(" ".join(tmp_res))  
        # print(tmp_res)

    return overlap_text_arr

# ---------------------여러 문장일경우-----------


# -----------------걍 한 컬럼별로 문장이 존재할경우 ------------

# file_name='test_data.csv'
# text=pd.read_csv(f'./{file_name}',encoding='cp949' ,index_col='num')
text=pd.read_csv('/gec_s/gec_s/gec_ktalk_92per_result.csv',
                 encoding='cp949' ,index_col='Unnamed: 0')

# -----------추론 시작 -------------
fin_result=[]
for i in trange(len(text)):
    text_num=text['form'][i]
    split_sentences(text_num)
    text_arr=text_num.split(' ')
    overlap_text_arr = no_overlap(text_arr,gram_num)
    overlap_text_arr = list(dict.fromkeys(overlap_text_arr)) # 중복제거
    overlap_text_arr.remove('')
    # print(overlap_text_arr)


    inf_overlap_text_arr = []
    for texts in overlap_text_arr:
        inputs = texts
        inputs = tokenizer([inputs], max_length=args.max_input_length,
                           truncation=True, return_tensors="pt").to(device)
        
        output = model.generate(**inputs, num_beams=4,
                                do_sample=False, max_length=args.max_input_length)
        
        # inputs = tokenizer([inputs], truncation=True, padding='max_length', max_length=args.max_input_length, return_tensors="pt").to(device)
        # output = model.generate(**inputs, max_length=args.max_input_length, num_beams=4, num_return_sequences=2, temperature=1.5, do_sample=True)

        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        inf_overlap_text_arr.append(decoded_output)
        # print(decoded_output)

    result=' '.join(inf_overlap_text_arr)
    fin_result.append(result)

 10%|████▌                                        | 1/10 [00:18<02:44, 18.31s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

 20%|█████████                                    | 2/10 [00:23<01:24, 10.51s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

 30%|█████████████▌                               | 3/10 [00:33<01:13, 10.55s/it]

...

 80%|████████████████████████████████████         | 8/10 [01:05<00:13,  6.98s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

 90%|████████████████████████████████████████▌    | 9/10 [01:11<00:06,  6.66s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

100%|████████████████████████████████████████████| 10/10 [01:17<00:00,  7.74s/it]


In [13]:
text[f'pko_t5_{gram_num}gram_form']=fin_result
text

,form,corrected_form,t5_form,busan_univ,pko_t5_5gram_form
0,기존점을 쓰고 그것에 관한 관점인가 감상인가 아닌가 이런 걸 구별하는 연습도 해보셔...,기준점을 주고 그것에 관한 관점인가 감상인가 아닌가 이런 걸 구별하는 연습도 해 보...,기존점을 쓰고 그것에 관한 관점인가 감상인가 아닌가 이런 걸 구별하는 연습도 해보셔...,NaN,기존점을 쓰고 그것에 관한 관점인가 감상인가 아닌가 이런 걸 구별하는 연습도 해보셔...
1,킬부도 시원을 하면 안 돼요,필부도 시건을 하면 안 돼요.,킬부도 시원을 하면 안 돼요?,NaN,킬부도 시원을 하면 안 돼요
2,현강의 문이 발짝 열셨습니다 서 일 이 강 얻어셨나요 복수를 잘 하셨나요 저 일강을...,삼강의 문이 활짝 열렸습니다. 자 일 이강 어떠셨나요? 복습 잘 하셨나요? 자 일강...,현강의 문이 발짝 열셨습니다. 서 일 이 강 얻어셨나요? 복수를 잘 하셨나요? 저 ...,NaN,현강의 문이 발짝 열셨다 서 일 이 강 얻어셨나요 복수를 잘 하셨나요 저 일강을 통...
3,화자가 누군가에게 말을 걷낸다는건 내 말을 들어주는 사람이 있는 거죠 그래서 이걸 ...,화자가 누군가에게 말을 건넨다는 건 내 말을 들어주는 사람이 있는 거죠. 그래서 이...,화자가 누군가에게 말을 걷낸다는 건 내 말을 들어주는 사람이 있는 거죠. 그래서 이...,NaN,화자가 누군가에게 말을 걷낸다는건 내 말을 들어주는 사람이 있는 거죠 그래서 이걸 ...
4,길게 써있는데요 아까 제목에서 봤던 거가 굉장히 비슷한 내용기죠,길게 쓰여있는데요. 아까 제목에서 봤던 것과 굉장히 비슷한 내용이죠.,길게 써있는데요. 아까 제목에서 봤던 거가 굉장히 비슷한 내용이죠. 세,NaN,길게 써있는데요 아까 제목에서 봤던 거가 굉장히 비슷한 내용기죠
5,계속 여러 번 나오고 있어요 이거 쉬 안 좋은 문제점이거든요 그것 을 보안하기위에 ...,계속 여러 번 나오고 있어요. 이것이 안 좋은 문제점이거든요. 그것을 보완하기 위해...,계속 여러 번 나오고 있어요. 이거 쉬 안 좋은 문제점이거든요. 그것 을 보완하기 ...,NaN,계속 여러 번 나오고 있어요 이거 쉬 안 좋은 문제점이거든요 그것 을 보안하기위에 ...
6,그다음 기자 기자의 질문 어린 석은 사람을 높이 평가이 이것을 긍정적으로 바라봤죠 ...,그다음 기자 기자의 질문 어리석은 사람을 높이 평가한 이유 이것을 긍정적으로 바라봤...,그다음 기자? 기자의 질문. 어린 석은 사람을 높이 평가. 이것을 긍정적으로 바라봤...,NaN,그다음 점 점의 질문 어린 석은 사람을 높이 평가이 이것을 긍정적으로 바라봤죠. 그...
7,자 반달리즘에 경향을 어떻게 이해한다고 했죠 그래서 뭔가 기존의 것에서 벗어나려고 ...,자 반달리즘의 경향을 어떻게 이해한다고 했죠 그렇죠? 뭔가 기존의 것에서 벗어나려고...,자 반달리즘에 경향을 어떻게 이해한다고 했죠? 그래서 뭔가 기존의 것에서 벗어나려고...,NaN,자 반달리즘에 경향을 어떻게 이해한다고 했죠. 그래서 뭔가 기존의 것에서 벗어나려고...
8,칼처럼 비정한 것이기 때문입니다 지문에 있었던 내용 그대로 가지고 왔거든요 적절하구요,칼처럼 비정한 것이기 때문입니다. 지문에 있었던 내용 그대로 가지고 왔거든요 적절하고요.,칼처럼 비정한 것이기 때문입니다. 지문에 있었던 내용 그대로 가지고 왔거든요. 적절...,NaN,칼처럼 비정한 것이기이기에. 지문에 있었던 내용 그대로 가지고 왔거든요 적절하구요.
9,교과서에 나와 있는 지문이 이것보다 훨씬 길고 이 앞에 작품이 하나 더 있어요 그죠,교과서에 나와 있는 지문이 이것보다 훨씬 길고 이 앞에 작품이 하나 더 있어요 그죠.,교과서에 나와 있는 지문이 이것보다 훨씬 길고 이 앞에 작품이 하나 더 있어요. 그죠?,NaN,교과서에 나와 있는 지문이 이것보다 훨씬 길고 이 앞에 작품이 하나 더 있어요. 그죠?


In [49]:
""" 추론 풀코드 baseline"""

# text=pd.read_csv('/gec_s/gec_s/gec_ktalk_result.csv', encoding='cp949' ,index_col='Unnamed: 0')
text=pd.read_csv('/gec_s/gec_s/gec_ktalk_92per_result.tsv',
                 sep='\t', encoding='utf-8' ,index_col='Unnamed: 0')

text=text.head(100)

# -----------추론 시작 -------------
fin_result=[]

for texts in tqdm(text['form'].tolist()):
    inputs = texts
    inputs = tokenizer([inputs], max_length=args.max_input_length,
                       truncation=True, return_tensors="pt").to(device)

    output = model.generate(**inputs, num_beams=4,
                            do_sample=False, max_length=args.max_input_length)

    # inputs = tokenizer([inputs], truncation=True, padding='max_length', max_length=args.max_input_length, return_tensors="pt").to(device)
    # output = model.generate(**inputs, max_length=args.max_input_length, num_beams=4, num_return_sequences=2, temperature=1.5, do_sample=True)

    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    fin_result.append(decoded_output)

fin_result[:5]

100%|████████████████████████████████████████████████████████████████████████| 100/100 [01:40<00:00,  1.01s/it]


['잘못했어요. 잘못한 건 아무것도 없어요. 정말이에요?',
 '대신에 공원이나 길거리를 가다 보면 그늘이 너무 없어요.',
 '이미 체결되고 장 마감됐는데요. 뭐 그러게 알지도 못하시면서 왜 손드세요.',
 '요즘에 가성비 갑인 제품이 얼마나 많은데',
 '얘가 이 가위를 빌려도 될까요?',
 '어떻게 하면 좋은 점수를 받을 수 있는 걸까?',
 '영진 씨 노래 자란다는 얘기 들었는데 축하드려요.',
 '좋은 기회가 될 거라고 생각해요.',
 '너는 현재 무슨 책임을 맡고 있어?',
 '실수한 사람은 바로 너야.',
 '네. 저도 처음에는 이 정도로 심할 줄 몰랐어요.',
 '사진을 어떻게 게시하는지 모르겠어.',
 '무엇이 중요한지 말씀해 보세요.',
 '오늘 학교 끝나고 아이스크림 먹으러 가서',
 '애들 크는 거 보면 정말 시간이 빠르다는 말이 실감 나.',
 '이어보세요. 너 지금 뭐 하고 있어?',
 '약속을 화, 수요일에서 수요일로 바꾸고 싶어요.',
 '옆집에서 새로 이사 오셨다고? 오신 떡이야?',
 '음 시간 나는 대로 연락 좀 해줄래?',
 '아니 십 일 월에 한 번 투진해 보자고.',
 '얀마 승연이가 지금 큰 날 보고 있어. 오늘 우리 아빠가 더 급해.',
 '화장실을 청소하려면 우선 청소용품을 먼저 사야 해.',
 '오후의 아이를 유치환에서 데려왔어요.',
 '좋아요. 시상식은 경찰서에 일 층에서 있을 거예요.',
 '왜요? 너 돈 찾으러 온 거 아니야?',
 '우리 엄마는 나에 대해 너무 자세히 알고 있어.',
 '이걸 볼 때마다 웃지 않을 수가 없어.',
 '저는 뭐에 십 그란도',
 '아 물론 지금은 그렇게 생각은 안 하지만 말이야.',
 '바로 내 이야기 같다는 느낌이 시청자들을 끌어들이고 있는 거죠.',
 '키보드  두드리는 소리가 너무 시끄러워.',
 '갑자기 장님이 됐지만 다른 모든 감각이 아주 예민해지는 기분이었어.',
 '할머니가 옥수수랑 감자 보내주셨어.',
 '지금 생각해보면 정말 세상 암울한 생각이었죠.',
 '난

In [50]:
text[f'pko_t5_form']=fin_result
text

,form,corrected_form,acc,pko_t5_form
0,잘못했어요 흠친 권 아무 것도 없어요 정말이에요,잘못했어요 훔친 건 아무것도 없어요 정말이에요.,85.71,잘못했어요. 잘못한 건 아무것도 없어요. 정말이에요?
1,대신에 공원이나 길거리를 가다보면 그늘이 너무 없어요,대신에 공원이나 길거리를 가다 보면 그늘이 너무 없어요.,95.83,대신에 공원이나 길거리를 가다 보면 그늘이 너무 없어요.
2,이미 체결되고 장마감 됐는데요 뭐 그러게 알지도 못하시면서 왜 손되세요,이미 체결되고 장 마감됐는데요 뭐 그러게 알지도 못해면서 왜 손대세요?,86.67,이미 체결되고 장 마감됐는데요. 뭐 그러게 알지도 못하시면서 왜 손드세요.
3,요즘에 가성비 갑인 제품이 얼마나 많은데,요즘에 가성비 갑인 제품이 얼마나 많은데?,94.44,요즘에 가성비 갑인 제품이 얼마나 많은데
4,얘가 이 가위를 빌려도 될까요,내가 이 가위를 빌려도 될까요?,84.62,얘가 이 가위를 빌려도 될까요?
...,...,...,...,...
95,나는 사진을 찍으면 못 생기게 나와,나는 사진을 찍으면 못생기게 나와.,93.33,나는 사진을 찍으면 못 생기게 나와.
96,혹시 몇 시에 출발했는지 알 수 있을까요,혹시 몇 시에 출발했는지 알 수 있을까요?,94.12,혹시 몇 시에 출발했는지 알 수 있을까요?
97,등살은 운동회야 빠지는 거지,등살은 운동해야 빠지는 거지?,84.62,등살은 운동해야 빠지는 거지.
98,옷 가방 책 필기구 말구 뭐가 더 필요한데,옷 가방 책 필기구 말고 뭐가 더 필요한데?,88.24,"옷, 가방, 책, 필기구 말구 뭐가 더 필요한데?"


In [51]:

def cal_cer(input_text: str, output_text: str) -> float:
    input_text = input_text.replace(' ', '')
    output_text = output_text.replace(' ', '')
    
    dist = Lev.distance(output_text, input_text)
    length = len(input_text)
    
    # assert length==0 ,'시발'
    return dist / length


# label_data=['기준점을 주고 그것에 관한 관점인가 감상인가 아닌가 이런 걸 구별하는 연습도 해 보셔야 합니다.']
# results=['기존점을 쓰고 그것에 관한 관점인가 감상인가 아닌가 이런 걸 구별하는 연습도 해보셔야 합니다.']


# cer 측정
def cal_acc(results:list, label_data:list) -> float:
    total_cer = 0.0
    for naver_data, answer_data in zip(results, label_data):
        cer = cal_cer(answer_data, naver_data)
        total_cer += cer

    result_cer = total_cer / len(results)
    acc=(1-result_cer) * 100
    return round(acc,2)

# print("Test CER: {:.3f}".format(result_cer))
# print("Test ACC: {:.3f}".format((1-result_cer) * 100))
# print(cal_acc(results, label_data))

lis=[]
for i in trange(len(text)):
    
    label_data=[text['corrected_form'][i]]
    results=[text[f'pko_t5_form'][i]]
    # results=[text[f'pko_t5_5gram_form'][i]]

    # print('1 :',label_data,results)
    lis.append(cal_acc(results, label_data))
    # print('2:',lis)
text['pko_t5_gec_acc']=lis

100%|█████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 17997.44it/s]


In [52]:
# a='기준점을 주고 그것에 관한 관점인가 감상인가 아닌가 이런 걸 구별하는 연습도 해 보셔야 합니다.'
# b='기존점을 쓰고 그것에 관한 관점인가 감상인가 아닌가 이런 걸 구별하는 연습도 해보셔야 합니다.'
# # a=a.replace(' ', '')
# # b=b.replace(' ', '')
# dist = Lev.distance(b, a)
# length = len(a)
# dist / length

In [53]:
text

,form,corrected_form,acc,pko_t5_form,pko_t5_gec_acc
0,잘못했어요 흠친 권 아무 것도 없어요 정말이에요,잘못했어요 훔친 건 아무것도 없어요 정말이에요.,85.71,잘못했어요. 잘못한 건 아무것도 없어요. 정말이에요?,71.43
1,대신에 공원이나 길거리를 가다보면 그늘이 너무 없어요,대신에 공원이나 길거리를 가다 보면 그늘이 너무 없어요.,95.83,대신에 공원이나 길거리를 가다 보면 그늘이 너무 없어요.,100.00
2,이미 체결되고 장마감 됐는데요 뭐 그러게 알지도 못하시면서 왜 손되세요,이미 체결되고 장 마감됐는데요 뭐 그러게 알지도 못해면서 왜 손대세요?,86.67,이미 체결되고 장 마감됐는데요. 뭐 그러게 알지도 못하시면서 왜 손드세요.,83.33
3,요즘에 가성비 갑인 제품이 얼마나 많은데,요즘에 가성비 갑인 제품이 얼마나 많은데?,94.44,요즘에 가성비 갑인 제품이 얼마나 많은데,94.44
4,얘가 이 가위를 빌려도 될까요,내가 이 가위를 빌려도 될까요?,84.62,얘가 이 가위를 빌려도 될까요?,92.31
...,...,...,...,...,...
95,나는 사진을 찍으면 못 생기게 나와,나는 사진을 찍으면 못생기게 나와.,93.33,나는 사진을 찍으면 못 생기게 나와.,100.00
96,혹시 몇 시에 출발했는지 알 수 있을까요,혹시 몇 시에 출발했는지 알 수 있을까요?,94.12,혹시 몇 시에 출발했는지 알 수 있을까요?,100.00
97,등살은 운동회야 빠지는 거지,등살은 운동해야 빠지는 거지?,84.62,등살은 운동해야 빠지는 거지.,92.31
98,옷 가방 책 필기구 말구 뭐가 더 필요한데,옷 가방 책 필기구 말고 뭐가 더 필요한데?,88.24,"옷, 가방, 책, 필기구 말구 뭐가 더 필요한데?",76.47


In [54]:
# 결과보고할땐 이거 엑셀에서 피벗시켜서 저장하믄되긋네
text.to_csv(f'./{args.output_dir}/pko_t5_92per_result.csv',sep=',',encoding='cp949')
# text.to_csv(f'./{args.output_dir}/pko_t5_5gram_result.csv',sep=',',encoding='cp949')


In [57]:
text[text['pko_t5_gec_acc']>=90].count()

form              66
corrected_form    66
acc               66
pko_t5_form       66
pko_t5_gec_acc    66
dtype: int64

In [ ]:
# d3=pd.read_csv(f'./3gram_test_data.csv',encoding='cp949')
# d4=pd.read_csv(f'./4gram_test_data.csv',encoding='cp949')
# d5=pd.read_csv(f'./5gram_test_data.csv',encoding='cp949')

In [ ]:
# d3=d3[['form','t5_3gram_form','acc']]
# d4=d4[['form','t5_4gram_form','acc']]
# d5=d5[['form','t5_5gram_form','acc']]

In [ ]:
d4

In [ ]:
data=pd.merge(d3,d4,on='form').merge(d5)

In [ ]:
data

In [ ]:
data[['form','t5_3gram_form','t5_4gram_form','t5_5gram_form','acc_x','acc_y','acc']].to_csv(f'./total_resultlist.csv',encoding='cp949')